# Clean and process data

## Import libraries

In [12]:
import pandas as pd
import geopandas as gpd
import numpy as np
import json

## Generate and process Congressional District geometry

In [13]:
#read in file
#https://geodata.bts.gov/datasets/usdot::congressional-districts/explore
gdf_0 = gpd.read_file('data/NTAD_Congressional_Districts_1928759543695387395.geojson')


#add columns as needed
gdf_0['FIPSNUM'] = gdf_0['STATEFP'].astype(int)


#create new GDF filtered on voting districts (still includes D.C.)
gdf_0 = gdf_0[(gdf_0['FIPSNUM'] < 57)&(gdf_0['FIPSNUM'] != 3)&(gdf_0['FIPSNUM'] != 7)
    &(gdf_0['FIPSNUM'] != 14)&(gdf_0['FIPSNUM'] != 43)&(gdf_0['FIPSNUM'] != 52)]


#validate CRS and that the GDF is still a valid coverage
print(gdf_0.crs)
print(gdf_0.is_valid_coverage())


#simplify the geometry for loading speed
#this will generate a SettingWithCopyWarning, but this is behaving as intended
gdf_0['geometry'] = gdf_0.loc[:, 'geometry'].simplify_coverage(0.005).buffer(0)

EPSG:4326
True


## Generate and process SNAP data

In [14]:
#read in file
#https://www.fns.usda.gov/sites/default/files/data-files/cat-snap-congressional-district-data-export.xlsx
sdf = pd.read_excel('data/cat-snap-congressional-district-data-export.xlsx', dtype={'fipscd': str, 'fips': str})


#set FIPS codes as strings for merge
sdf['fipscd'] = sdf['fipscd'].astype('string')
sdf['fips'] = sdf['fips'].astype('string')


#merge SNAP data into GeoDataFrame
gdf_s = gdf_0.merge(sdf, how='left', left_on='GEOID', right_on='fipscd', validate='1:1')

## Generate and process election data

In [15]:
#read in file
#https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/IG0UN2
#returns a mixed type warning, but we are dropping the column in question later anyways
edf = pd.read_csv('data/1976-2024-house.tab')


#filter to most recent midterms, create match key, and rename column so there isn't any confusion in merge
edf = edf[edf['year'] == 2022]
edf['GEOID'] = edf['state_fips'].astype(str).str.zfill(2) + edf['district'].astype(str).str.zfill(2)
edf = edf.rename(columns={'party': 'party_delete'})


#clean up for merge
columns_remove_1 = ['year', 'state', 'state_po', 'state_fips', 'state_cen', 'state_ic', 'office', 'district', 'stage', 'runoff', 
                    'special', 'candidate', 'writein', 'mode', 'totalvotes', 'unofficial', 'version', 'fusion_ticket']

edf = edf.drop(columns = columns_remove_1)


#merge election data into GeoDataFrame
#because of its format, we need to do two sweeps of the election data to populate two different columns
gdf_prefips = gdf_s.merge(edf[edf['party_delete'] == 'REPUBLICAN'].groupby('GEOID').sum(), how='left', on='GEOID', validate='1:1')
gdf_prefips = gdf_prefips.rename(columns={'candidatevotes': 'R votes'})

gdf_prefips = gdf_prefips.merge(edf[edf['party_delete'] == 'DEMOCRAT'].groupby('GEOID').sum(), how='left', on='GEOID', validate='1:1')
gdf_prefips = gdf_prefips.rename(columns={'candidatevotes': 'D votes'})


#clean up and reduce size
columns_remove_2 = ['LSAD', 'CDSESSN', 'MTFCC', 'FUNCSTAT', 'OFFICE_ID', 'BIOGUIDE_ID', 'OFFICE_AUDIT_ID', 'PREFIX', 'FIRSTNAME', 
       'MIDDLENAME', 'LASTNAME', 'SUFFIX', 'LISTING_NAME', 'PHONE', 'WEBSITEURL', 'VACANT', 'CONTACTFORMURL', 'PHOTOURL', 'FACE_BOOK_URL', 
       'TWITTER_URL', 'YOUTUBE_URL', 'INSTAGRAM_URL', 'FLICKR_URL', 'VACANCY', 'ROOM_NUM', 'HOB', 'COMMITTEE_ASSIGNMENTS', 'LAST_UPDATED', 
       'hh_poverty_pct_in_snap', 'hh_race_aian_pct_in_snap', 'hh_race_asian_pct_in_snap', 'hh_race_black_pct_in_snap',
       'hh_race_hisp_pct_in_snap', 'hh_race_nhpi_pct_in_snap', 'hh_race_other_pct_in_snap', 'hh_race_twoplus_pct_in_snap', 
       'hh_race_white_pct_in_snap', 'hh_60plus_pct', 'hh_snap_60plus', 'hh_snap_60plus_pct', 'hh_snap_children', 'hh_snap_children_pct', 
       'hh_snap_disabled', 'hh_snap_disabled_pct', 'hh_snap_poverty', 'hh_snap_poverty_pct', 'hh_snap_race_aian', 'hh_snap_race_aian_pct', 
       'hh_snap_race_asian', 'hh_snap_race_asian_pct', 'hh_snap_race_black', 'hh_snap_race_black_pct', 'hh_snap_race_hisp', 
       'hh_snap_race_hisp_pct', 'hh_snap_race_nhpi', 'hh_snap_race_nhpi_pct', 'hh_snap_race_other', 'hh_snap_race_other_pct', 
       'hh_snap_race_twoplus', 'hh_snap_race_twoplus_pct', 'hh_snap_race_white', 'hh_snap_race_white_pct', 'hh_total_60plus', 
       'hh_total_disabled', 'hh_total_disabled_pct', 'hh_total_poverty', 'hh_total_poverty_pct', 'hh_total_race_aian', 
       'hh_total_race_aian_pct', 'hh_total_race_asian', 'hh_total_race_asian_pct', 'hh_total_race_black', 'hh_total_race_black_pct', 
       'hh_total_race_hisp', 'hh_total_race_hisp_pct', 'hh_total_race_nhpi', 'hh_total_race_nhpi_pct', 'hh_total_race_other', 
       'hh_total_race_other_pct', 'hh_total_race_twoplus', 'hh_total_race_twoplus_pct', 'hh_total_race_white', 'hh_total_race_white_pct', 
       'medhhinc', 'party_delete_x', 'party_delete_y']

gdf_prefips = gdf_prefips.drop(columns = columns_remove_2)


#zero out NaNs and recalculate PARTY
gdf_prefips['R votes'] = gdf_prefips['R votes'].fillna(0)
gdf_prefips['D votes'] = gdf_prefips['D votes'].fillna(0)

gdf_prefips['PARTY'] = np.where(gdf_prefips['D votes'] > gdf_prefips['R votes'], 'D', 'R')
gdf_prefips['PARTYNUM'] = np.where(gdf_prefips['PARTY'] == 'R', 1, -1)


#sort for pie chart
gdf_prefips = gdf_prefips.sort_values(by=['PARTY', 'hh_snap'])

C:\Users\Connor\AppData\Local\Temp\ipykernel_9300\2849011101.py:3: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  edf = pd.read_csv('data/1976-2024-house.tab')


## Create FIPS mapping

In [24]:
#import base state data and clean
states_df = pd.read_csv('https://www2.census.gov/geo/docs/reference/codes2020/national_state2020.txt', sep='|')
states_df = states_df[states_df['STATEFP'] < 57]
states_df['FIPS'] = states_df['STATEFP'].astype(str).str.zfill(2)
states_df = states_df.drop(columns = ['STATE', 'STATEFP', 'STATENS'])


#create additional geographies
usa_df = pd.DataFrame({'STATE_NAME': ['All U.S.', '---STATES---'], 'FIPS': ['USA', '--']})
regions_df = pd.DataFrame({'STATE_NAME': ['---REGIONS---', 'Northeast Region', 'Midwest Region', 'South Region', 'West Region'], 'FIPS': ['--', 'R01', 'R02', 'R03', 'R04']})
divs_df = pd.DataFrame({'STATE_NAME': ['---DIVISIONS---', 'New England Div.', 'Middle Atlantic Div.', 'East North Central Div.', 'West North Central Div.', 'South Atlantic Div.', 'East South Central Div.', 'West South Central Div.', 'Mountain Div.', 'Pacific Div.'], 'FIPS': ['--', 'D01', 'D02', 'D03', 'D04', 'D05', 'D06', 'D07', 'D08', 'D09']})


#append and save out fips_df for use in pie chart title
fips_df = pd.concat([usa_df, states_df, regions_df, divs_df], ignore_index=True)
fips_df.to_csv('data/fips.csv', index=False)


#set region and division code lists for each state to be merged in
reg_codes = pd.Series(['--', '--','R03', '--', 'R04', 'R03', 'R04', 'R04', 'R01', 'R03', 'R03', 'R03', 'R03', '--', 'R04', 'R02', 'R02', 'R02', 'R02', 'R03', 'R03', 'R01', 'R03', 'R01', 'R02', 'R02', 'R03', 'R02', 'R04', 'R02', 'R04', 'R01', 'R01', 'R04', 'R01', 'R03', 'R02', 'R02', 'R03', 'R04', 'R01', 'R01', 'R03', 'R02', 'R03', 'R03', 'R04', 'R01', 'R03', 'R04', 'R03', 'R02', 'R04'], copy=False)
div_codes = pd.Series(['--', '--','D06', '--', 'D08', 'D07', 'D09', 'D08', 'D01', 'D05', 'D05', 'D05', 'D05', '--', 'D08', 'D03', 'D03', 'D04', 'D04', 'D06', 'D07', 'D01', 'D05', 'D01', 'D03', 'D04', 'D06', 'D04', 'D08', 'D04', 'D08', 'D01', 'D02', 'D08', 'D02', 'D05', 'D04', 'D03', 'D07', 'D09', 'D02', 'D01', 'D05', 'D04', 'D06', 'D07', 'D08', 'D01', 'D05', 'D09', 'D05', 'D03', 'D08'], copy=False)


#append region/division columns to fips_df
fips_df['reg_codes'] = reg_codes
fips_df['div_codes'] = div_codes
fips_df = fips_df.fillna('--')


#merge in region/division codes to gdf
gdf_final = gdf_prefips.merge(fips_df[fips_df['FIPS'] != '--'], how='left', left_on='STATEFP', right_on='FIPS', validate='m:1')

## Save out processed data as GeoJSON

In [25]:
gdf_final.to_file('data/processed.geojson', driver='GeoJSON')